# Models

> Collection of Deep Learning models

In [ ]:
#| default_exp models

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export
import torch.nn.functional as F
import torch.nn as nn
import torch
from nimrod.modules import Encoder, Decoder
import pytorch_lightning as pl

/home/syl20/anaconda3/envs/nimrod/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
class AutoEncoder(nn.Module):
    def __init__(self,
        encoder:Encoder, # Encoder layer
        decoder:Decoder # Decoder layer
        ):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self,
        x:torch.Tensor # Tensor B x L
        )->torch.Tensor:
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

In [ ]:
enc = Encoder()
dec = Decoder()
a = AutoEncoder(enc, dec)
batch = torch.rand((10, 28*28))
y = a(batch)
print(y.shape)

torch.Size([10, 784])


In [ ]:
from nimrod.data.datasets import MNISTDataset
from torch.utils.data import DataLoader

ds = MNISTDataset()
dl = DataLoader(ds)
b = next(iter(dl))
print(len(b), b[0].shape, b[1].shape)


2 torch.Size([1, 28, 28]) torch.Size([1])


In [ ]:
#| export
class AutoEncoderPL(pl.LightningModule):
    def __init__(self, autoencoder:AutoEncoder):
        super().__init__()
        self.save_hyperparameters(ignore=['autoencoder'])
        self.autoencoder = autoencoder

    def forward(self, x):
        return self.autoencoder(x)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1) # flatten B x C x H x W to B x L (grey pic)
        x_hat = self.autoencoder(x)
        loss = F.mse_loss(x_hat, x)
        self.log("train_loss", loss)
        return loss
    
    def testing_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1) # flatten B x C x H x W to B x L (grey pic)
        x_hat = self.autoencoder(x)
        loss = F.mse_loss(x_hat, x)
        self.log("test_loss", loss, on_step=True, on_epoch=True, sync_dist=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1) # flatten B x C x H x W to B x L (grey pic)
        x_hat = self.autoencoder(x)
        loss = F.mse_loss(x_hat, x)
        self.log("val_loss", loss, on_step=True, on_epoch=True, sync_dist=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()